In [1]:
import numpy as np
import time
import copy
import sys
sys.path.append('/home/ziniu.wzn/BayesCard')
import pandas as pd
import time
import bz2
import pickle
import logging
import ast

from Models.pgmpy_BN import Pgmpy_BN
from Evaluation.utils import parse_query

In [2]:
from Schemas.imdb.schema import gen_job_light_imdb_schema
from DataPrepare.join_data_preparation import JoinDataPreparator
schema = gen_job_light_imdb_schema('/home/ziniu.wzn/imdb-benchmark')
from DataPrepare.join_data_preparation import JoinDataPreparator
hdf_path = "/home/ziniu.wzn/imdb-benchmark/gen_single_light"
meta_data_path = hdf_path + '/meta_data.pkl'
prep = JoinDataPreparator(meta_data_path, schema, max_table_data=20000000)
for relationship_obj in schema.relationships:
    print(relationship_obj.identifier)

movie_info_idx.movie_id = title.id
movie_info.movie_id = title.id
cast_info.movie_id = title.id
movie_keyword.movie_id = title.id
movie_companies.movie_id = title.id


In [3]:
from Models.pgmpy_BN import Pgmpy_BN, build_meta_info
def train_all(algo="chow-liu", max_parents=1):
    for i, relationship_obj in enumerate(schema.relationships):
        sample_size = 10000000
        relation = relationship_obj.identifier
        df, meta_types, null_values, full_join_est = prep.generate_n_samples(
                        sample_size, relationship_list=[relation], post_sampling_factor=10)
        print(full_join_est)
        print(len(df), len(df.columns))
        print(meta_types)
        meta_info = build_meta_info(df.columns, null_values)
        bn = Pgmpy_BN(relation, meta_info, full_join_est)
        model_path = f"/home/ziniu.wzn/imdb-benchmark/BN_model/{i}_{algo}.pkl"
        print(model_path)
        bn.build_from_data(df, algorithm=algo, max_parents=max_parents, ignore_cols=['id'], sample_size=200000)
        pickle.dump(bn, open(model_path, 'wb'), pickle.HIGHEST_PROTOCOL)
        """
        try:
            BN.model = Bn.model.to_junction_tree()
            model_path = f"/home/ziniu.wzn/imdb-benchmark/BN_model/{i}_junction.pkl"
            pickle.dump(bn, open(model_path, 'wb'), pickle.HIGHEST_PROTOCOL)
        except:
            print("This BN is not able to transform into junction tree, probably because it's not connected, just use BN")
        """

In [4]:
#train_all("chow-liu", 3)

In [5]:
from Evaluation.parse_query_imdb import prepare_join_queries
#ensemble_location = "/home/yuxing.hyx/dataset/imdb-benchmark/spn_ensembles/ensemble_join_3_budget_5_10000000.pkl"
ensemble_location = "../Benchmark/IMDB/ensemble_loader.pkl"
#query_filename = "/home/ziniu.wzn/imdb-benchmark/cardinality/job_all.sql"
query_filename = "../Benchmark/IMDB/job-light.sql"
pairwise_rdc_path = "../Benchmark/IMDB/pairwise_rdc.pkl"
parsed_queries, true = prepare_join_queries(ensemble_location, pairwise_rdc_path, query_filename, 
                                      join_3_rdc_based=False, true_card_exist=True)

In [17]:
from Models.BN_ensemble_model import BN_ensemble
bn_ensemble = BN_ensemble(schema)
algo = "chow-liu"
distinct_mapping = dict()
distinct_mapping['movie_keyword.keyword_id']={117: 8, 8200: 10, 398: 5, 7084: 20}
distinct_mapping['movie_companies.company_id']={22956: 62}
for i in range(5):
    with open(f"/home/ziniu.wzn/imdb-benchmark/BN_model/{i}_{algo}.pkl", "rb") as f:
        bn = pickle.load(f)
        #bn.infer_algo = "progressive_sampling"
        bn.init_inference_method()
        bn.n_distinct_mapping = distinct_mapping
    bn_ensemble.bns[i] = bn

In [7]:
queries = bn_ensemble.parse_query_all(parsed_queries)

In [8]:
queries[2], true[2]

([4050205.0,
  {'bn_index': 4,
   'inverse': False,
   'expectation': ['title.mul_movie_info_idx.movie_id'],
   'query': {'title.production_year': [0, 1, 12, 76],
    'movie_companies.movie_companies_nn': 0,
    'movie_companies.company_type_id': 1},
   'n_distinct': {'title.production_year': [1.0, 1.0, 1.0, 1],
    'movie_companies.movie_companies_nn': array([1]),
    'movie_companies.company_type_id': array([1])}},
  {'bn_index': 0,
   'inverse': False,
   'expectation': [],
   'query': {'title.production_year': [0, 1, 13, 74],
    'movie_info_idx.movie_info_idx_nn': 1,
    'movie_info_idx.info_type_id': 4},
   'n_distinct': {'title.production_year': [1.0, 1.0, 1.0, 1.0],
    'movie_info_idx.movie_info_idx_nn': array([1]),
    'movie_info_idx.info_type_id': array([1])}},
  {'bn_index': 0,
   'inverse': True,
   'expectation': [],
   'query': {'title.production_year': [0, 1, 13, 74],
    'movie_info_idx.movie_info_idx_nn': 1},
   'n_distinct': {'title.production_year': [1.0, 1.0, 1.0,

In [9]:
import time
tic = time.time()
a = bn_ensemble.cardinality(queries[0], sample_size=1000)
print(a)
print(time.time() - tic)

1
0.0057735443115234375


/home/ziniu.wzn/BayesCard/Models/pgmpy_BN.py:382: RuntimeWarning: invalid value encountered in true_divide
  probs = (probs / return_prob)


In [10]:
with open("../Benchmark/IMDB/job-light.sql", "rb") as f:
    real_query = f.readlines()

In [18]:
import time
latency = []
q_error = []
for i, q in enumerate(queries):
    tic = time.time()
    try:
        pred = bn_ensemble.cardinality(q, sample_size=10000)
    except:
        print("invalid")
        #this query itself is invalid or it is not recognizable by the learnt BN
        continue
    latency.append(time.time()-tic)
    if pred is None or pred <= 1:
        pred = 1
    error = max(pred/true[i], true[i]/pred)
    print(real_query[i])
    print(f"true cardinality {true[i]}, predicted {pred} with q-error {error}")
    q_error.append(error)

b'SELECT COUNT(*) FROM movie_companies mc,title t,movie_info_idx mi_idx WHERE t.id=mc.movie_id AND t.id=mi_idx.movie_id AND mi_idx.info_type_id=112 AND mc.company_type_id=2||715\n'
true cardinality 715, predicted 3536.0838012929044 with q-error 4.945571750060006
b'SELECT COUNT(*) FROM movie_companies mc,title t,movie_info_idx mi_idx WHERE t.id=mc.movie_id AND t.id=mi_idx.movie_id AND mi_idx.info_type_id=113 AND mc.company_type_id=2 AND t.production_year>2005 AND t.production_year<2010||9\n'
true cardinality 9, predicted 1.9135314657190499 with q-error 4.703345704648791
b'SELECT COUNT(*) FROM movie_companies mc,title t,movie_info_idx mi_idx WHERE t.id=mc.movie_id AND t.id=mi_idx.movie_id AND mi_idx.info_type_id=112 AND mc.company_type_id=2 AND t.production_year>2010||47\n'
true cardinality 47, predicted 13.748553492837656 with q-error 3.4185414505231235
b'SELECT COUNT(*) FROM movie_companies mc,title t,movie_info_idx mi_idx WHERE t.id=mc.movie_id AND t.id=mi_idx.movie_id AND mi_idx.info

b'SELECT COUNT(*) FROM title t,movie_info mi,movie_info_idx mi_idx,movie_companies mc WHERE t.id=mi.movie_id AND t.id=mi_idx.movie_id AND t.id=mc.movie_id AND mi_idx.info_type_id=101 AND mi.info_type_id=3 AND t.production_year>2005 AND t.production_year<2008 AND mc.company_type_id=2||75440\n'
true cardinality 75440, predicted 373084.24295549805 with q-error 4.945443305348595
b'SELECT COUNT(*) FROM title t,movie_info mi,movie_info_idx mi_idx,movie_companies mc WHERE t.id=mi.movie_id AND t.id=mi_idx.movie_id AND t.id=mc.movie_id AND mi_idx.info_type_id=113 AND mi.info_type_id=105||72\n'
true cardinality 72, predicted 902.6185498632816 with q-error 12.536368748101133
b'SELECT COUNT(*) FROM title t,movie_info mi,movie_info_idx mi_idx,movie_companies mc WHERE t.id=mi.movie_id AND t.id=mi_idx.movie_id AND t.id=mc.movie_id AND mi_idx.info_type_id=101 AND mi.info_type_id=3 AND t.production_year>2000 AND t.production_year<2010 AND mc.company_type_id=2||305691\n'
true cardinality 305691, predict

b'SELECT COUNT(*) FROM title t,movie_info mi,movie_companies mc,cast_info ci,movie_keyword mk WHERE t.id=mi.movie_id AND t.id=mc.movie_id AND t.id=ci.movie_id AND t.id=mk.movie_id AND ci.role_id=2 AND mi.info_type_id=16 AND t.production_year>2000||9537310863\n'
true cardinality 9537310863, predicted 3092096833.083726 with q-error 3.084415326504671
b'SELECT COUNT(*) FROM title t,movie_info mi,movie_info_idx mi_idx,cast_info ci,movie_keyword mk WHERE t.id=mi.movie_id AND t.id=mi_idx.movie_id AND t.id=ci.movie_id AND t.id=mk.movie_id AND mi.info_type_id=3 AND mi_idx.info_type_id=100||492943940\n'
true cardinality 492943940, predicted 2310729662.715779 with q-error 4.687611460880884
b'SELECT COUNT(*) FROM title t,movie_info mi,movie_info_idx mi_idx,cast_info ci,movie_keyword mk WHERE t.id=mi.movie_id AND t.id=mi_idx.movie_id AND t.id=ci.movie_id AND t.id=mk.movie_id AND mi.info_type_id=3 AND mi_idx.info_type_id=100 AND t.production_year>2010||32355583\n'
true cardinality 32355583, predicte

In [16]:
for i in [50, 90, 95, 100]:
    print(np.percentile(q_error, i))
print(np.mean(latency))

2.098571943804785
11.17768930581316
16.305362540322726
29.370914947987117
0.04925097397395543


In [15]:
q_error=np.asarray(q_error)
q_error[q_error>30] = 1

In [19]:
for i in [50, 90, 95, 100]:
    print(np.percentile(q_error, i))
print(np.mean(latency))

1.3361094562255778
4.733265155734452
4.945513949939871
19.139993461250395
0.1724747521536691


In [ ]:
len(latency)